In [5]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [6]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,6.686620,True,18,1,43.0
1,9.360828,True,32,1,41.0
2,8.437091,False,30,1,39.0
3,6.124442,False,24,1,40.0
4,7.125340,False,26,1,41.0


In [7]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [8]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [9]:
cat ~/.ssh/id_rsa.pub

ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQCfEb/yBpcgRyqawD06aVDtk23ALAVbsL2obqgNdqSGCZMXYypmkekxjEoY1jUmzC8T9MaHVVaIaYM1mH5dNJhxxtgklsX2lmaaOXO6/uzuNG9gTC1Jl5AClRutdCmD+k1mDB7+OCVGDtCm5oI/LJBraEWUSh0mUUez4Z+AP5zWM/LZEpHdOxIDkCek/dSaGlfDVVooGPrMldn2KAStgJZSKHIUj0gF5VjyZsRLwjgTJSTg2LwtmHCUYpjuQfW+zBvaUhXYJSXtMj8n/l0Ci57E1r5egqiIlw2Bk7jCBz5Wngapqv+sSfRguM6C7IxtjagXS9GvB6kkBXUTt82Lcifmtx8u2yoE7SLG2rV0tkA3cdL85IPn0nszfd7fx3aKCgzmPKKfh5emWcxlyiNI6lrZtVBGoODCQeI/sQjV1BT/ndhGEGfiIeg/2tTHE3862v0mb1T1eZ9SqoK+ob0zV+yJhKrCL2YfGe0/NmfPen38DJTY+VTFQ+GDr9MDjIbGvDU6Khu/02VAeNlnjyXIeEeWREo/rTZMHxXEcAXeS31fsfFdNhjTanJERxbBx12xd2R37HhpMpto9EtrYxzEkzBXGA9WIMeteF02ZLOS5W/9Q5PsUyxuwjwAE6Qtj0N0u+MkkRd3ZPDCN7rwJy54a5CRLKO3FUxh/3QRk7ruPDcZeQ== marcacohen@gmail.com
